In [98]:
import ixmp as ix
import message_ix
import warnings
warnings.filterwarnings('ignore')

mp = ix.Platform('local',jvmargs=["-Xmx8G"])

In [99]:
scen_og = message_ix.Scenario(mp, model="MESSAGEix-GLOBIOM", scenario="ENGAGE_SSP2_EN_NPi2020_500")
#scen = scen_og.clone(model="MESSAGEix-GLOBIOM" , scenario="ENGAGE_SSP2_EN_NPi2020_500_zero_vre_integration_constraints", keep_solution=False)
scen = scen_og.clone(model="MESSAGEix-GLOBIOM" , scenario="ENGAGE_SSP2_EN_NPi2020_500_removing_IC_and_curtailment", keep_solution=False)
scen.check_out()

Remove the curtailment constraints:

In [100]:
def remove_curtailment(scen, df_rel_activity, source=None):
    if source is None:
        df_curtailment = df_rel_activity.loc[df_rel_activity.relation.str.contains("curtailment")]
        tech = [x for x in scen.set("technology") if "curtailment" in x]
        df_curtailment_techs = df_curtailment.loc[df_curtailment.technology.str.contains("curtailment")]
        print("removing all curtailment")
    else:
        df_curtailment = df_rel_activity.loc[df_rel_activity.relation.str.contains(source + "_curtailment")]
        tech = [x for x in scen.set("technology") if source + "_curtailment" in x]
        df_curtailment_techs = df_curtailment.loc[df_curtailment.technology.str.contains("curtailment")]
        print("removing " + source + " curtailment")
        
    scen.remove_par("relation_activity", df_curtailment_techs)

    df_inputs = scen.par('input',{"technology":tech})
    scen.remove_par("input",df_inputs)

    df_inputs.value = 0
    scen.add_par("input", df_inputs)

Remove the renewable integration costs:

In [101]:
def remove_integration_cost(scen):
    df_var_cost = scen.par("var_cost")
    df_integration_cost = df_var_cost.loc[df_var_cost.technology.str.contains("cv")]
    scen.remove_par("var_cost", df_integration_cost)
    df_integration_cost.value = 0
    scen.add_par("var_cost", df_integration_cost)

Add slack to the firm capacity constraint:

In [102]:
def remove_firm_capacity_constraint(scen, df_rel_activity):
    df_res_marg = df_rel_activity.query("relation == 'res_marg'")
    df_res_marg_cv = df_res_marg.loc[df_res_marg.technology.str.contains("cv")]
    scen.remove_par("relation_activity", df_res_marg_cv)
    df_res_marg_cv.value = 1
    scen.add_par("relation_activity", df_res_marg_cv)

Add slack to the flexibility constraint:

In [103]:
def remove_flexibility_constraint(scen, df_rel_activity):
    df_oper_res = df_rel_activity.query("relation == 'oper_res'")
    df_oper_res_cv = df_oper_res.loc[df_oper_res.technology.str.contains("cv")]
    scen.remove_par("relation_activity", df_oper_res_cv)
    df_oper_res_cv.value = 0
    scen.add_par("relation_activity", df_oper_res_cv)

Remove only the desired constraints:

In [104]:
df_rel_activity = scen.par("relation_activity")
remove_curtailment(scen, df_rel_activity)
# remove_firm_capacity_constraint(scen, df_rel_activity)
# remove_flexibility_constraint(scen, df_rel_activity)
remove_integration_cost(scen)

removing all curtailment


Commit and solve:

In [84]:
scen.commit("remove curtailment and integration costs")

In [85]:
scen.solve(solve_options={"lpmethod": "4",
                          "advind":"0",
                         "lpmethod":"4","threads":"4", 
                         "epopt":"1e-06",
                         "scaind":"-1"})

In [1]:
mp.close_db()

NameError: name 'mp' is not defined